# Update the Key ,Value, Type in Postgresql 

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import psycopg2,os,csv

from IPython.display import display
pd.options.display.max_columns = None

conn = psycopg2.connect("dbname=postgres user=postgres password=Msjobs@123")


def insert_data(key_,value_,type_):
    cur = conn.cursor()
    sql_command = "INSERT INTO public.lct_ml_key_value(key_,value_,type_) VALUES('{}','{}','{}');".format(str(key_), str(value_),str(type_))
    cur.execute(sql_command)
    conn.commit()
    cur.close() 
    
def csv_to_DB(df):
    test_df  = pd.DataFrame()
    test_df1 = pd.DataFrame()

    test_df['data']   = df[df.columns[0]].replace("'","").astype(str)
    test_df['result'] = df.columns[0] #[test.columns[0],test.columns[0]] 

    for i in df.columns[1:]:
        test_df1['data'] = df[i].astype(str).str.replace("'","")
        test_df = test_df.append(test_df1,ignore_index=True)
        test_df.result.fillna(i.replace(" ", "_"),inplace=True)
    test_df = test_df[test_df.data!='nan']
    
    return test_df

def DB_insert(test_df,type_):
    for i in range(len(test_df)):
        key_ = test_df.iloc[i][1]
        value_ = test_df.iloc[i][0]
        insert_data(key_,value_,type_)

# flask app

In [2]:
from flask import Flask, request, render_template, session, redirect

app = Flask(__name__)

@app.route('/')
def my_form():
    return render_template('DB_update/DB_update.html')

@app.route('/', methods=("POST", "GET"))
def html_table():
    if request.method == 'POST':
        try:
            myfile = request.form['myfile']
            type_ = request.form['type_']
            print(myfile,type_)
            
            file_name = 'Train_Data/'+myfile
            df_csv = pd.read_csv(file_name)
            df = csv_to_DB(df_csv)
#             df = df.drop_duplicates()
            DB_insert(df,type_)

            test_df =pd.DataFrame(df.result.value_counts())
            test_df1 = df.groupby('result').first()
        except ValueError:
            return "Please check if the values are entered correctly"

    return render_template('DB_update/Inserted_Data.html',  tables=[test_df.to_html(classes='data')],tables1=[test_df1.to_html(classes='data')])

if __name__ == '__main__':
    app.run(host = '127.0.0.1', port='5001',threaded=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Jul/2020 17:24:19] "GET / HTTP/1.1" 200 -


location.csv location


127.0.0.1 - - [13/Jul/2020 17:24:55] "POST / HTTP/1.1" 200 -


# Testing

In [2]:
# def select_data():
#     cur = conn.cursor()
#     sql_command = "select * from public.lct_ml_key_value;"
#     data = pd.read_sql(sql_command, conn)
#     conn.commit()
#     cur.close()
#     return data    

# def select_data_type(type_):
#     cur = conn.cursor()
#     sql_command = "select * from public.lct_ml_key_value where type_ ='"+type_+"'"
#     data = pd.read_sql(sql_command, conn)
#     conn.commit()
#     cur.close()
#     return data 

# df = pd.read_csv("Train_Data/sites.factory2020-07-04T02_44_21.921.csv")

In [6]:
# test_df = csv_to_DB(df)
# type_ = "location"
# DB_insert(test_df,type_)

In [1]:
# from flask import Flask, request, render_template, session, redirect
# ### working code -- which displays the DataFrame

# app = Flask(__name__)

# @app.route('/')
# def my_form():
#     return render_template('DB_update_select.html')

# @app.route('/', methods=("POST", "GET"))
# def html_table():
#     if request.method == 'POST':
#         try:
#             type_ = request.form['type_']
#             key_ = request.form['key_']
#             value_ = request.form['value_']
#             print(type_,key_,value_)
#             df = select_data_type(type_)
#         except ValueError:
#             return "Please check if the values are entered correctly"

#     return render_template('Predict_Confidence.html',  tables=[df.to_html(classes='data')])

# if __name__ == '__main__':
#     app.run()

In [ ]:
# Key_ = '' #Insert the key which is a category in my case 
# Value_ '' # Insert the value for the corresponding category 
# Tye_ = '' # Type of data i.e., Location ,Purchase order

# insert_data(Key_,Value_,Tye_)

In [1]:
# df = select_data()
# df.head()

In [2]:
# type_list = set(df.type_)
# type_list

In [1]:
# df_type_based = select_data_type('location')
# df_type_based.head()

In [15]:
# pd.DataFrame(csv_to_DB().result.value_counts())

# csv_to_DB().groupby('result').first()